In [2]:
from bs4 import BeautifulSoup
import csv
import pandas as pd
import requests
import time
import os

In [3]:
#htmls = ['acceleratorPhysics.html', 'adaptationAndSelf-OrganizingSystems.html', 'algebraicGeometry.html', 'algebraicTopology.html', 'analysisOfPDEs.html', 'applications.html', 'appliedPhysics.html', 'artificialIntelligence.html', 'astrophysicsOfGalaxies.html', 'atmosphericAndOceanicPhysics.html', 'atomicAndMolecularClusters.html', 'atomicPhysics.html', 'audioAndSpeechProcessing.html', 'biologicalPhysics.html', 'biomolecules.html', 'categoryTheory.html', 'cellBehavior.html', 'cellularAutomataAndLatticeGases.html', 'chaoticDynamics.html', 'chemicalPhysics.html', 'classicalAnalysisAndODEs.html', 'classicalPhysics.html', 'combinatorics.html', 'commutativeAlgebra.html', 'complexVariables.html', 'computation.html', 'computationalComplexity.html', 'computationalEngineering,Finance,AndScience.html', 'computationalFinance.html', 'computationalGeometry.html', 'computationalPhysics.html', 'computationAndLanguage.html', 'computersAndSociety.html', 'computerScience.html', 'computerScienceAndGameTheory.html', 'computerVisionAndPatternRecognition.html', 'cosmologyAndNongalacticAstrophysics.html', 'cryptographyAndSecurity.html', 'dataAnalysis,StatisticsAndProbability.html', 'databases.html', 'dataStructuresAndAlgorithms.html', 'differentialGeometry.html', 'digitalLibraries.html', 'discreteMathematics.html', 'disorderedSystemsAndNeuralNetworks.html', 'distributed,Parallel,AndClusterComputing.html', 'dynamicalSystems.html', 'earthAndPlanetaryAstrophysics.html', 'econometrics.html', 'economics.html', 'electricalEngineeringAndSystemsScience.html', 'emergingTechnologies.html', 'exactlySolvableAndIntegrableSystems.html', 'fluidDynamics.html', 'formalLanguagesAndAutomataTheory.html', 'functionalAnalysis.html', 'generalEconomics.html', 'generalFinance.html', 'generalLiterature.html', 'generalMathematics.html', 'generalPhysics.html', 'generalRelativityAndQuantumCosmology.html', 'generalTopology.html', 'genomics.html', 'geometricTopology.html', 'geophysics.html', 'graphics.html', 'groupTheory.html', 'hardwareArchitecture.html', 'highEnergyAstrophysicalPhenomena.html', 'highEnergyPhysics-Experiment.html', 'highEnergyPhysics-Lattice.html', 'highEnergyPhysics-Phenomenology.html', 'highEnergyPhysics-Theory.html', 'historyAndOverview.html', 'historyAndPhilosophyOfPhysics.html', 'human-ComputerInteraction.html', 'imageAndVideoProcessing.html', 'informationRetrieval.html', 'informationTheory.html', 'instrumentationAndDetectors.html', 'instrumentationAndMethodsForAstrophysics.html', 'K-TheoryAndHomology.html', 'logic.html', 'logicInComputerScience.html', 'machineLearning.html', 'materialsScience.html', 'mathematicalFinance.html', 'mathematicalPhysics.html', 'mathematicalSoftware.html', 'mathematics.html', 'medicalPhysics.html', 'mesoscaleAndNanoscalePhysics.html', 'methodology.html', 'metricGeometry.html', 'molecularNetworks.html', 'multiagentSystems.html', 'multimedia.html', 'networkingAndInternetArchitecture.html', 'neuralAndEvolutionaryComputing.html', 'neuronsAndCognition.html', 'nonlinearSciences.html', 'nuclearExperiment.html', 'nuclearTheory.html', 'numberTheory.html', 'numericalAnalysis.html', 'operatingSystems.html', 'operatorAlgebras.html', 'optics.html', 'optimizationAndControl.html', 'otherComputerScience.html', 'otherCondensedMatter.html', 'otherQuantitativeBiology.html', 'otherStatistics.html', 'patternFormationAndSolitons.html', 'performance.html', 'physics.html', 'physicsAndSociety.html', 'physicsEducation.html', 'plasmaPhysics.html', 'popularPhysics.html', 'populationsAndEvolution.html', 'portfolioManagement.html', 'pricingOfSecurities.html', 'probability.html', 'programmingLanguages.html', 'quantitativeBiology.html', 'quantitativeFinance.html', 'quantitativeMethods.html', 'quantumAlgebra.html', 'quantumGases.html', 'quantumPhysics.html', 'representationTheory.html', 'ringsAndAlgebras.html', 'riskManagement.html', 'robotics.html', 'signalProcessing.html', 'socialAndInformationNetworks.html', 'softCondensedMatter.html', 'softwareEngineering.html', 'solarAndStellarAstrophysics.html', 'sound.html', 'spacePhysics.html', 'spectralTheory.html', 'statisticalFinance.html', 'statisticalMechanics.html', 'statistics.html', 'statisticsTheory.html', 'stronglyCorrelatedElectrons.html', 'subcellularProcesses.html', 'superconductivity.html', 'symbolicComputation.html', 'symplecticGeometry.html', 'systemsAndControl.html', 'theoreticalEconomics.html', 'tissuesAndOrgans.html', 'tradingAndMarketMicrostructure.html']

In [4]:
def clear_arxiv_file():
    if os.path.exists('arxiv_data.csv'):
        with open('arxiv_data.csv', 'w') as file:
            pass
        with open('arxiv_data.csv', 'a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['title','abstract','day','month','year'])

In [10]:
def get_all_main_links():
    request = requests.get("https://arxiv.org/")
    if (request.status_code == 200):
        soup = BeautifulSoup(request.content, 'html.parser')
        all_main = soup.find('div', id='content').find_all('ul')
        all_main = all_main[:-1]
        returns_list = []
        for e in all_main:
            a = e.find_all('a')
            a = [tag for tag in a if tag.text.lower() not in ['recent', 'search', 'detailed description']]
            for link in a:
                returns_list.append(link['href'])

    return returns_list

In [6]:
def month_to_int(month_abbr):
    # Dictionary mapping month abbreviations to integers
    month_dict = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
        'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
        'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }
    # Convert the month abbreviation to its corresponding integer
    return month_dict.get(month_abbr, None)

In [15]:
links = get_all_main_links()
time.sleep(1)
clear_arxiv_file()
for link in links:
    if(link[1:].split('/')[0] == 'archive'):
        continue
    request = requests.get("https://arxiv.org" + link)
    soup = BeautifulSoup(request.content, 'html.parser')
    inside_files = [x['href'] for x in soup.find('div', id='dlpage').find_all('a', title='Abstract')]
    for inside_file in inside_files:
        inside_req = requests.get("https://arxiv.org" + inside_file)
        inside_soup = BeautifulSoup(request.content, 'html.parser')
        #print(request.content)

        #data = soup.find('div', id='content-inner')
        #date = data.find('div', class_="dateline").get_text().strip()[1:-1].split(" ")

        #try:
        #    day = int(date[-3])
        #except:
        #    day = None

        #try:
        #    month = month_to_int(date[-2])
        #except:
        #    month = None
        #try:
        #    year = int(date[-1])
        #except:
        #    year = 2024
        
        #h1_tag = soup.find('h1', class_='title mathjax')
        #span_tag = h1_tag.find('span', class_='descriptor')
        #title = h1_tag.get_text(separator='', strip=True)
        #title = title.replace(span_tag.text, '')
        
        #h1_tag = soup.find('blockquote', class_='abstract mathjax')
        #abstract = h1_tag.get_text(separator='', strip=True)
        #if abstract.startswith("Abstract:"):
        #    abstract = abstract[len("Abstract:"):].strip()

        #with open('arxiv_data.csv', 'a', newline='', encoding='utf-8') as file:
        #    writer = csv.writer(file)
        #    writer.writerow([title, abstract, day, month, year])
        #    time.sleep(1000)

/abs/2405.04542
/abs/2405.04560
/abs/2405.04564
/abs/2405.04572
/abs/2405.04574
/abs/2405.04577
/abs/2405.04578
/abs/2405.04580
/abs/2405.04582
/abs/2405.04583
/abs/2405.04587
/abs/2405.04593
/abs/2405.04598
/abs/2405.04601
/abs/2405.04604
/abs/2405.04626
/abs/2405.04637
/abs/2405.04646
/abs/2405.04651
/abs/2405.04660
/abs/2405.04663
/abs/2405.04719
/abs/2405.04723
/abs/2405.04725
/abs/2405.04731
/abs/2405.04740
/abs/2405.04744
/abs/2405.04749
/abs/2405.04750
/abs/2405.04766
/abs/2405.04779
/abs/2405.04784
/abs/2405.04802
/abs/2405.04830
/abs/2405.04836
/abs/2405.04849
/abs/2405.04857
/abs/2405.04870
/abs/2405.04871
/abs/2405.04886
/abs/2405.04893
/abs/2405.04898
/abs/2405.04908
/abs/2405.04922
/abs/2405.04925
/abs/2405.04935
/abs/2405.04981
/abs/2405.05013
/abs/2405.05032
/abs/2405.05046
/abs/2405.05055
/abs/2405.05058
/abs/2405.05063
/abs/2405.05064
/abs/2405.05104
/abs/2405.05111
/abs/2405.05115
/abs/2405.05120
/abs/2405.05127
/abs/2405.05147
/abs/2405.05184
/abs/2405.05185
/abs/240

KeyboardInterrupt: 

In [ ]:
df = pd.read_csv("arxiv_data.csv")
df.head()

,'title','abstract','day','month','year'
0,German Beams -- The Story of Particle Accelera...,German Beams -- The Story of Particle Accelera...,6.0,5.0,2024
1,Observing S-Matrix Pole Flow in Resonance Inte...,Observing S-Matrix Pole Flow in Resonance Inte...,6.0,5.0,2024
2,Laser wakefield acceleration of ions with a tr...,Laser wakefield acceleration of ions with a tr...,4.0,5.0,2024
3,GAIA: A General AI Assistant for Intelligent A...,GAIA: A General AI Assistant for Intelligent A...,2.0,5.0,2024
4,From Compton Scattering of photons on targets ...,From Compton Scattering of photons on targets ...,1.0,5.0,2024
